In [28]:
import concurrent.futures
import requests

def make_completion_request(prompt):
    url = 'http://localhost:8000/v1/completions'
    headers = {'Content-Type': 'application/json'}
    data = {
        "model": "mistralai/Mistral-7B-Instruct-v0.2",
        "prompt": prompt,
        "max_tokens": 4096,
        "temperature": 0.7
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()

def make_several_completion_requests(prompts):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(make_completion_request, prompts)
    results = list(results)
    return results

prompts = ["Résume moi la vie de Babar", "Qu'est-ce que le Machine Learning?", "Explique le concept de récursivité"]


In [29]:
results = make_several_completion_requests(prompts)

In [31]:
results

[{'id': 'cmpl-3050137d84f546a2ae03ab6905899e06',
  'object': 'text_completion',
  'created': 429392,
  'model': 'mistralai/Mistral-7B-Instruct-v0.2',
  'choices': [{'index': 0,
    'text': ", un éléphant élégant et raffiné qui règne sur une grande forêt et qui a été élevé par une belle princesse humaine.\n\nBabar est un éléphant élégant et raffiné qui a été élevé par une belle princesse humaine dans sa jeunesse. Il règne maintenant sur une grande forêt composée d'éléphants, et il est connu pour sa gracieuse apparence et sa gentillesse.\n\nBabar est né dans la forêt des éléphants sauvages, mais il a été trouvé par la princesse humaine dans ses premiers jours de vie. Elle l'a adopté et l'a élevé comme un de ses propres enfants, lui enseignant les manières et les coutumes humaines. Babar a appris à parler, à lire et à écrire, et il a adopté les vêtements et les habits de la cour royale humaine.\n\nQuand Babar a grandi, il a décidé de retourner dans sa forêt natale pour devenir roi des élé

In [22]:
for res in results:
    print(res)

{'id': 'cmpl-43d379804c38441482401ab237c93466', 'object': 'text_completion', 'created': 429254, 'model': 'mistralai/Mistral-7B-Instruct-v0.2', 'choices': [{'index': 0, 'text': " l'éléphant\n\nBabar est un éléphant blanc né dans la forêt vierge. Il est élevé par sa mère, la reine des éléphants, qui l'apprête à devenir roi. Quand celle-ci est tuée par un chasseur, Babar est très triste et décide de quitter la forêt. Il rencontre une princesse rose et s'enfume. Il décide de s'installer à Paris, où il rencontre des humains. Il s'habille avec des vêtements humains et se voit accorder la main de la princesse. Ils se marient et Babar devient le roi de Paris. Ils ont deux enfants, Alexandre et Marie. Babar continue à aller dans sa forêt natale et à s'occuper de ses sujets éléphants. Il a également une tante, Mme Saxo, qui vit dans une tour et qui aide à élever ses enfants.\n\nBabar's Life, Summed Up\n\nBabar is a white elephant born in the forest. He is raised by his elephant queen mother, who

In [24]:
for res in results:
    print(res)

In [6]:
import pandas as pd
import pandas as pd
import numpy as np

# Load the vectorized database
df = pd.read_csv('../rag_db.csv')
df.embedding = df.embedding.apply(lambda x: eval(x))

def get_file_chunk(filename, df):
    chunks = df[df.filename == filename.split('.')[0]].text.tolist()
    return chunks

def group_chunks_by(chunks, nb):
    return [chunks[i:i+nb] for i in range(0, len(chunks), nb)]

chunks = get_file_chunk('youtube-2.mp3', df)
group_chunks_by(chunks, 8)


[[" Je vais vous présenter une entreprise créée par trois français, un ancien chercheur de DeepMind et deux qui étaient chez Meta.\n  Cette entreprise, qui n'existait pas il y a à peine huit mois, a eu le temps dans cet intervalle de faire trembler toute l'industrie de l'IA\n  en publiant des modèles alternatifs à ChatGPT qui explosent toute la concurrence, être valorisé à presque 2 milliards de dollars.\n  Le tout sans aucune communication, ni vidéo promotionnelle déceptive.\n  Rien.\n  Ce que fait cette boîte me hype tellement que je vais quasiment tous les jours sur Twitter exclusivement pour vérifier qu'ils n'ont pas fait des nouvelles annonces.\n  Et c'est véridique.\n  C'est vrai ?\n  Oui.\n  Laissez-moi vous expliquer à quel point nos petits français ont explosé le game et comment vous pourriez aussi en profiter.\n  Pour commencer, ce que je vous propose, c'est de regarder un tableau des meilleures intelligences artificielles qui sont concurrentes à ChatGPT.\n  Vous allez voir, 

In [45]:
import requests
from flask import Flask, render_template, redirect, url_for, request
import concurrent.futures
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Prompt
PROMPT = """Je vais t'envoyer un texte commencant et finissant par des guillemets.
- Tu es un secretaire.  
- On veut un compte rendu en markdown de ce texte tire de l'enregistrement d'une video. 

Instruction importantes:
- Ecrit entierement ta reponse en markdown.
- Tu dois ecrire en markdown.

Tes instructions :
- Structure l'information.
- Donne un titre au texte.
- Decoupe le texte en 2 ou 3 grandes parties en les numerotant et titre les.
- Decoupe les parties en sous partie en les numerorant.
- Fait un point pour chaque information.
- Fait des phrases avec un sujet, un verbe et un complement et un determinant pour les noms pour chaque point.
- Ecrit en francais.
- Si tu veux ecrire le texte en anglais, ecrit en francais.
- On veut que l'information soit structure.
- Ecrit entierement ta reponse en markdown, c'est important.
- Tu dois ecrire en markdown. 
- Ne repete pas les memes informations.
- Ne fait pas de conclusion.  
"""

# Load the model
model = SentenceTransformer('flaubert/flaubert_base_uncased')

# Load the vectorized database
df = pd.read_csv('../rag_db.csv')
df.embedding = df.embedding.apply(lambda x: eval(x))

def find_closest_chunk(query, df, model, n=3):
    query_embedding = model.encode([query])[0]
    similarity = df.embedding.apply(lambda x: cosine_similarity([query_embedding], [x])[0][0])
    ids = similarity.sort_values(ascending=False).head(n)
    return df.loc[ids.index].text.tolist()

def get_file_chunk(filename, df):
    chunks = df[df.filename == filename.split('.')[0]].text.tolist()
    return chunks

def group_chunks_by(chunks, nb):
    return [chunks[i:i+nb] for i in range(0, len(chunks), nb)]

def make_completion_request(prompt):
    url = 'http://localhost:8000/v1/completions'
    headers = {'Content-Type': 'application/json'}
    data = {
        "model": "mistralai/Mistral-7B-Instruct-v0.2",
        "prompt": prompt,
        "max_tokens": 4096,
        "temperature": 0.7
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()

def make_several_completion_requests(prompts):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(make_completion_request, prompts)
    results = list(results)
    return results

def make_summary(filename):
    # Retrieve chunks
    chunks = group_chunks_by(get_file_chunk(filename, df), 8)
    prompts = []
    for chunk in chunks:
        prompt = PROMPT + "\"" + ''.join(chunk) + "\"\n\n"
        prompts.append(prompt)
    results = make_several_completion_requests(prompts)
    summaries = [result['choices'][0]['text'] for result in results]

    result = make_completion_request(PROMPT + "\"" + ''.join(summaries) + "\"\n\n")
    return result['choices'][0]['text'] 

No sentence-transformers model found with name flaubert/flaubert_base_uncased. Creating a new one with MEAN pooling.
